In [1]:
!pip install kafka-python

/usr/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point
     |████████████████████████████████| 246 kB 1.3 MB/s eta 0:00:01


In [1]:
import json
import time 

from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

True

In [85]:
t0 = time.time()

topic_name = 'test-topic'

for i in range(10):
    message = {'number': i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)

t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')

producer.flush()

t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')

Sent: {'number': 0}
Sent: {'number': 1}
Sent: {'number': 2}
Sent: {'number': 3}
Sent: {'number': 4}
Sent: {'number': 5}
Sent: {'number': 6}
Sent: {'number': 7}
Sent: {'number': 8}
Sent: {'number': 9}
took 0.50 seconds
took 0.50 seconds


In [3]:
import pandas as pd

In [10]:
!wget -nc https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz

File ‘green_tripdata_2019-10.csv.gz’ already there; not retrieving.



In [4]:
df_green = pd.read_csv("green_tripdata_2019-10.csv.gz", usecols=[
    'lpep_pickup_datetime',
    'lpep_dropoff_datetime',
    'PULocationID',
    'DOLocationID',
    'passenger_count',
    'trip_distance',
    'tip_amount'
])

In [14]:
df_green.head

<bound method NDFrame.head of        lpep_pickup_datetime lpep_dropoff_datetime  PULocationID  DOLocationID  \
0       2019-10-01 00:26:02   2019-10-01 00:39:58           112           196   
1       2019-10-01 00:18:11   2019-10-01 00:22:38            43           263   
2       2019-10-01 00:09:31   2019-10-01 00:24:47           255           228   
3       2019-10-01 00:37:40   2019-10-01 00:41:49           181           181   
4       2019-10-01 00:08:13   2019-10-01 00:17:56            97           188   
...                     ...                   ...           ...           ...   
476381  2019-10-31 23:30:00   2019-11-01 00:00:00            65           102   
476382  2019-10-31 23:03:00   2019-10-31 23:24:00           129           136   
476383  2019-10-31 23:02:00   2019-10-31 23:23:00            61           222   
476384  2019-10-31 23:42:00   2019-10-31 23:56:00            76            39   
476385  2019-10-31 23:23:00   2019-10-31 23:56:00            56           215  

In [5]:
t0 = time.time()

topic_name = 'green-trips'

for row in df_green.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    # print(row_dict)

    producer.send(topic_name, value=row_dict)

producer.flush()

t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')

took 75.43 seconds


In [6]:
import pyspark
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

24/03/14 00:33:36 WARN Utils: Your hostname, ih-PC resolves to a loopback address: 127.0.1.1; using 192.168.1.103 instead (on interface enp2s0)
24/03/14 00:33:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/ih/workspace/dezoomcamp/spark-3.3.2-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ih/.ivy2/cache
The jars for the packages stored in: /home/ih/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d2058764-f485-4713-ad47-a428db6e0ae1;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.2 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution r

24/03/14 00:33:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

In [8]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

query = green_stream.writeStream.foreachBatch(peek).start()

24/03/14 00:34:27 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-b897b0e1-7035-4d8b-9b11-5aa057def5ec. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/14 00:34:28 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


[Row(key=None, value=bytearray(b'{"lpep_pickup_datetime": "2019-10-01 00:26:02", "lpep_dropoff_datetime": "2019-10-01 00:39:58", "PULocationID": 112, "DOLocationID": 196, "passenger_count": 1.0, "trip_distance": 5.88, "tip_amount": 0.0}'), topic='green-trips', partition=0, offset=0, timestamp=datetime.datetime(2024, 3, 14, 0, 31, 14, 245000), timestampType=0)]
Row(key=None, value=bytearray(b'{"lpep_pickup_datetime": "2019-10-01 00:26:02", "lpep_dropoff_datetime": "2019-10-01 00:39:58", "PULocationID": 112, "DOLocationID": 196, "passenger_count": 1.0, "trip_distance": 5.88, "tip_amount": 0.0}'), topic='green-trips', partition=0, offset=0, timestamp=datetime.datetime(2024, 3, 14, 0, 31, 14, 245000), timestampType=0)
[Row(lpep_pickup_datetime='2019-10-01 00:26:02', lpep_dropoff_datetime='2019-10-01 00:39:58', PULocationID=112, DOLocationID=196, passenger_count=1.0, trip_distance=5.88, tip_amount=0.0)]
Row(lpep_pickup_datetime='2019-10-01 00:26:02', lpep_dropoff_datetime='2019-10-01 00:39:

In [9]:
query.stop()

In [90]:
query.stop()

In [10]:
from pyspark.sql import types

schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())

In [11]:
from pyspark.sql import functions as F

green_stream = green_stream \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")

In [12]:
query = green_stream.writeStream.foreachBatch(peek).start()

24/03/14 00:35:41 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-cbcf1b32-5485-4b3e-97f0-56368ba88551. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/14 00:35:41 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [13]:
query.stop()

In [30]:
popular_destinations = green_stream.withColumn("timestamp", F.current_timestamp()).groupBy(F.window(F.col("timestamp"), "5 minutes"), F.col("DOLocationID")).count().orderBy(F.desc('count'))

In [29]:
popular_destinations

DataFrame[window: struct<start:timestamp,end:timestamp>, DOLocationID: int, count: bigint]

In [ ]:
query = popular_destinations \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()

24/03/14 00:58:44 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-22e39bf6-a715-4d51-a01a-3fa029d3288e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/14 00:58:44 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 00:55:00, 2024-03-14 01:00:00}|74          |17741|
|{2024-03-14 00:55:00, 2024-03-14 01:00:00}|42          |15942|
|{2024-03-14 00:55:00, 2024-03-14 01:00:00}|41          |14061|
|{2024-03-14 00:55:00, 2024-03-14 01:00:00}|75          |12840|
|{2024-03-14 00:55:00, 2024-03-14 01:00:00}|129         |11930|
|{2024-03-14 00:55:00, 2024-03-14 01:00:00}|7           |11533|
|{2024-03-14 00:55:00, 2024-03-14 01:00:00}|166         |10845|
|{2024-03-14 00:55:00, 2024-03-14 01:00:00}|236         |7913 |
|{2024-03-14 00:55:00, 2024-03-14 01:00:00}|223         |7542 |
|{2024-03-14 00:55:00, 2024-03-14 01:00:00}|238         |7318 |
|{2024-03-14 00:55:00, 2024-03-14 01:00:00}|82          |7292 |
|{2024-

In [ ]:
query.stop()